## Cleaning up Atlas data - IPSL REA
**Function**      : Preprocess netCDF files and restructure the dataset<br>
**Description**   : In this notebook serves to clean up Atlas data which is given in netcdf format and aggregate the data into a single file.<br>
**Return Values   : .nc files**<br>
**Note**          : All the data is saved to netCDF4 format. Note that data from different models may vary concerning the resolution and coordinates.<br>

In [1]:
import os
from pathlib import Path
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset

### Path
Specify the path to the dataset and the place to save the outputs. <br>

In [2]:
# please specify data path
datapath = './AtlasData/raw'

# please specify output path
output_path = './preprocess'
os.makedirs(output_path, exist_ok = True)

Components used to create the output file names. Here, only `institution_id` and `cmor_var` is based on on CMIP DRS conventions.

In [3]:
output_file_name = {
    "prefix": "atlas",
    "activity": "EUCP", # project name e.g. EUCP
    "institution_id": "IPSL", # IPSL
    "source": "CORDEX", # e.g. CMIP5, CMIP6 or CORDEX
    "method": "REA", # e.g. REA
    "sub_method": "cons", # e.g. cons or uncons
    "cmor_var": "tas", # e.g. tas or pr
}

In [4]:
# CORDEX mask nc file
MASK_PATH = f"{datapath}/IPSL_REA/region_mask_NEU-CEU-MED_land.nc"
dataset_mask = Dataset(MASK_PATH)

### Load and process raw data

Make some functions to combining multiple dimensions with a preprocessor and load data

In [5]:
def add_percentile(ds):
    filename = ds.encoding["source"]
    percentile = int(filename.split('_')[-2])
    dataset, variable, future, _, reference, percentile, season = filename.split('_')[-7:]
    return(ds
           .drop_vars('height', errors='ignore')
           .assign_coords(percentile=int(percentile)).expand_dims('percentile')
          )

# data loader and batch processing
def load_data(project, season, variable):
    # open multiple files with xarray
    ds = xr.open_mfdataset(str(Path(datapath, 'IPSL_REA', f'eur_{project}_{variable}_2041-2060_vs_1995-2014_*_{season}.nc')),
                       preprocess=add_percentile)
    weighted = ds[f'{variable}_weighted'].rename(variable).assign_coords(constrained=1).expand_dims('constrained')
    unweighted = ds[f'{variable}_unweighted'].rename(variable).assign_coords(constrained=0).expand_dims('constrained')    
    return xr.concat([weighted, unweighted], dim='constrained')

Call functions

In [6]:
ipsl_rea_ds = {}
for source in ["CMIP5", "CMIP6", "CORDEX"]:
    seasons = []
    for season in ['DJF', 'JJA']:
        tas = load_data(source, season, 'tas')
        pr = load_data(source, season, 'pr')
        ds = xr.merge([tas, pr]).assign_coords(season=season)
        seasons.append(ds)
    ipsl_rea_ds[source] = xr.concat(seasons, dim='season')
ipsl_rea_ds

{'CMIP5': <xarray.Dataset>
 Dimensions:      (lon: 20, lat: 17, percentile: 5, constrained: 2, season: 2)
 Coordinates:
   * lon          (lon) float64 -8.75 -6.25 -3.75 -1.25 ... 33.75 36.25 38.75
   * lat          (lat) float64 31.25 33.75 36.25 38.75 ... 66.25 68.75 71.25
   * percentile   (percentile) int64 10 25 50 75 90
   * constrained  (constrained) int64 1 0
   * season       (season) <U3 'DJF' 'JJA'
 Data variables:
     tas          (season, constrained, percentile, lat, lon) float64 dask.array<chunksize=(1, 1, 1, 17, 20), meta=np.ndarray>
     pr           (season, constrained, percentile, lat, lon) float64 dask.array<chunksize=(1, 1, 1, 17, 20), meta=np.ndarray>,
 'CMIP6': <xarray.Dataset>
 Dimensions:      (lon: 20, lat: 17, percentile: 5, constrained: 2, season: 2)
 Coordinates:
   * lon          (lon) float64 -8.75 -6.25 -3.75 -1.25 ... 33.75 36.25 38.75
   * lat          (lat) float64 31.25 33.75 36.25 38.75 ... 66.25 68.75 71.25
   * percentile   (percentile) int64 10

Make some metadata. Here, we follow CF-conventions as much as possible.

In [7]:
attrs = {"tas" : {"description":"Change in Air Temperature",
            "standard_name":"Change in Air Temperature",
            "long_name": "Change in Near-Surface Air Temperature",
            "units": "K", # in line with raw data
            "cell_methods":"time: mean changes over 20 years 2041-2060 vs 1995-2014",
           },
         "pr" : {"description":"Relative precipitation",
            "standard_name":"Relative precipitation",
            "long_name": "Relative precipitation",
            "units": "%", # in line with raw data
            "cell_methods":"time: mean changes over 20 years 2041-2060 vs 1995-2014",
           },
         "latitude": {"units": "degrees_north", "long_name": "latitude", "axis": "Y"},
         "longitude": {"units": "degrees_east", "long_name": "longitude", "axis": "X"},
         "time": {"climatology": "climatology_bounds",
                  "long_name": "time",
                  "axis": "T",
                  "climatology_bounds": ['2050-6-1', '2050-9-1', '2050-12-1', '2051-3-1'],
                  "description": "mean changes over 20 years 2041-2060 vs 1995-2014. The mid point 2050 is chosen as the representative time."},
         "percentile": {"units": "%", "long_name": "percentile", "axis": "Z"},
}
                

### Assemble data and save to netcdf

Make a function to assemble and save data 

In [17]:
TIMES =  {"JJA": "2050-7-16", "DJF": "2051-1-16"} # "0000-4-16", "0000-7-16", "0000-10-16", "0000-1-16" MAM JJA SON DJF
PERCENTILES = [10, 25, 50, 75, 90]

def assembly(project, ds_original, var, cons):
    """
    Select data from original nc files and save the target fields.
    """
    if project == "CORDEX":
        LAT = dataset_mask.variables['lat'][:, 0]
        LON = dataset_mask.variables['lon'][0, :]
    else:
        LAT = ds_original.coords['lat'].values
        LON = ds_original.coords['lon'].values
    ds_target = xr.Dataset(
                {var: (("time", "latitude", "longitude","percentile"),
                 np.full([len(TIMES), len(LAT), len(LON), len(PERCENTILES)], np.nan)),
                "climatology_bounds": (pd.to_datetime(['2050-6-1', '2050-9-1', '2050-12-1', '2051-3-1']))},
                coords={    
                        "time": pd.to_datetime(list(TIMES.values())),
                        "latitude": LAT,
                        "longitude": LON, 
                        "percentile": PERCENTILES,
                 },
                 attrs={"description": "Contains modified IPSL REA data used for Atlas in EUCP project.",
                       "history": f"original IPSL REA data files eur_{project}_{var}_2041-2060_vs_1995-2014_*_DJF.nc, eur_{project}_{var}_2041-2060_vs_1995-2014_*_JJA.nc"}
    )
    for season in ["JJA", "DJF"]:
        for j, p in enumerate(PERCENTILES):
            ds_target[var].values[list(TIMES).index(season),:,:,j] = ds_original[var].sel(percentile=p, season=season, constrained=cons).values
    return ds_target

Call the function

In [18]:
for source in ["CMIP5", "CMIP6", "CORDEX"]:
    output_file_name["source"] = source
    for VAR_NAME in ["tas", "pr"]:
        output_file_name["cmor_var"] = VAR_NAME 
        for projection_ind, projection in enumerate(["uncons","cons"]):
            output_file_name["sub_method"] = projection
            new_ds = assembly(source, ipsl_rea_ds[source], VAR_NAME, projection_ind)

            # Fix attributes
            for key in new_ds.keys():
                new_ds[key].attrs = attrs[key]

            file_name = f"{'_'.join(output_file_name.values())}.nc"
            print(f"one dataset is saved to {file_name}")
            new_ds.to_netcdf(os.path.join(output_path, file_name))

one dataset is saved to atlas_EUCP_IPSL_CMIP5_REA_uncons_tas.nc
one dataset is saved to atlas_EUCP_IPSL_CMIP5_REA_cons_tas.nc
one dataset is saved to atlas_EUCP_IPSL_CMIP5_REA_uncons_pr.nc
one dataset is saved to atlas_EUCP_IPSL_CMIP5_REA_cons_pr.nc
one dataset is saved to atlas_EUCP_IPSL_CMIP6_REA_uncons_tas.nc
one dataset is saved to atlas_EUCP_IPSL_CMIP6_REA_cons_tas.nc
one dataset is saved to atlas_EUCP_IPSL_CMIP6_REA_uncons_pr.nc
one dataset is saved to atlas_EUCP_IPSL_CMIP6_REA_cons_pr.nc
one dataset is saved to atlas_EUCP_IPSL_CORDEX_REA_uncons_tas.nc
one dataset is saved to atlas_EUCP_IPSL_CORDEX_REA_cons_tas.nc
one dataset is saved to atlas_EUCP_IPSL_CORDEX_REA_uncons_pr.nc
one dataset is saved to atlas_EUCP_IPSL_CORDEX_REA_cons_pr.nc


### Check output

Load one of the saved data.